In [1]:
import random
import numpy as np 
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')
%matplotlib inline

### 추천 데이터 로딩
ratings = pd.read_csv('~/dev/research/RecommendSystem/data/the-movies-dataset/ratings_small.csv')
ratings.head()
# ratings.describe()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
### 영화 데이터 로딩
meta = pd.read_csv('~/dev/research/RecommendSystem/data//the-movies-dataset/movies_metadata.csv', dtype={'popularity': str})
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
### 데이터 정제
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return '|'.join(genres_list)

meta = meta[['id', 'adult', 'budget', 'genres', 'original_language', 'original_title', 'revenue', 'vote_average', 'vote_count']]
# meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta.head()
meta = meta.rename(columns={'id':'movieId'})
meta = meta.rename(columns={'original_title':'title'})
meta.head()
# meta = meta[meta['original_language'] == 'en']
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
meta['genres'] = meta['genres'].apply(parse_genres)
data = pd.merge(ratings, meta, on='movieId', how='inner')
data.head()

,userId,movieId,rating,timestamp,adult,budget,genres,original_language,title,revenue,vote_average,vote_count
0,1,1371,2.5,1260759135,False,17000000,Drama,en,Rocky III,270000000.0,6.6,894.0
1,4,1371,4.0,949810302,False,17000000,Drama,en,Rocky III,270000000.0,6.6,894.0
2,7,1371,3.0,851869160,False,17000000,Drama,en,Rocky III,270000000.0,6.6,894.0
3,19,1371,4.0,855193404,False,17000000,Drama,en,Rocky III,270000000.0,6.6,894.0
4,21,1371,3.0,853852263,False,17000000,Drama,en,Rocky III,270000000.0,6.6,894.0


In [8]:
### 특정 한명의 최고 점수 영화중 한개를 random 을 통해 추출
choose_user_id = 2
choose_movie = ''

user = data[data['userId'] == choose_user_id]
user_max = user[user['rating'] == user['rating'].max()]
user_max.head()

user_random_index = random.randint(0, len(user_max.index) -1)
user_random = user.iloc[[user_random_index]]
user_random.head()
choose_movie_title = user_random.iloc[0]['title']
choose_movie_title
choose_movie_id = user_random.iloc[0]['movieId']
choose_movie_id

150

In [14]:
matrix = data.pivot_table(index='userId', columns='title', values='rating')
matrix.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...Più forte ragazzi!,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,...,사마리아,야수,연가시,오직 그대만,올드보이,"장화, 홍련",최종병기 활,친절한 금자씨,해안선,활
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
def pearson(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def pearson_collaboration_filter(choose_movie_title, matrix, n):
    result = []
    for mn in matrix.columns:
        # 선택 영화 필터
        if mn == choose_movie_title:
            continue
        
        # 기존에 본 영화 필터
        
        # pearson correlation
        cor = pearson(matrix[choose_movie_title], matrix[mn])
        
        if np.isnan(cor):
            continue
        else:
            result.append((mn, '{:.2f}'.format(cor)))
            
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

# matrix.columns
### content base 를 통해 Pearson correlation 상위 10개의 가장 가까운 영화 찾기
pearson_collaboration_filter_result = pearson_collaboration_filter(choose_movie_title, matrix, 100)
pearson_collaboration_filter_result

/Users/whitexozu/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in double_scalars


[('The Hours', '0.29'),
 ('Crustacés et coquillages', '0.24'),
 ('When Saturday Comes', '0.23'),
 ('Les Choristes', '0.22'),
 ('Adventures Of A Taxi Driver', '0.20'),
 ('Ah ! Les belles bacchantes', '0.20'),
 ('Aliens vs Predator: Requiem', '0.20'),
 ('Big Fish', '0.20'),
 ('Boogie Nights', '0.20'),
 ('Carry On Cleo', '0.20'),
 ('Get Carter', '0.20'),
 ('Lolita', '0.20'),
 ('Rain Man', '0.20'),
 ('The Wedding Planner', '0.20'),
 ('Armageddon', '0.19'),
 ('Ask the Dust', '0.19'),
 ('Memoirs of an Invisible Man', '0.19'),
 ('...Più forte ragazzi!', '0.18'),
 ('Batman Returns', '0.18'),
 ('Die Höhle des gelben Hundes', '0.18'),
 ('Falstaff', '0.18'),
 ('Ludwig', '0.18'),
 ('Milano Rovente', '0.18'),
 ('My Name Is Bruce', '0.18'),
 ('Reservoir Dogs', '0.18'),
 ('The Golden Compass', '0.18'),
 ('Caché', '0.17'),
 ('Club Paradise', '0.17'),
 ('Der letzte Mann', '0.17'),
 ('Faraon', '0.17'),
 ('Say Anything...', '0.17'),
 ('The Green Mile', '0.17'),
 ('座頭市海を渡る', '0.17'),
 ('Amico, stammi lont

In [34]:
meta[meta['movieId'] == choose_movie_id]

,movieId,adult,budget,genres,original_language,title,revenue,vote_average,vote_count
4919,150.0,False,12000000,Thriller|Action|Comedy|Crime|Drama,en,48 Hrs.,78868508.0,6.5,364.0


In [64]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(A, B):
    A_C = np.nan_to_num(np.array(A.tolist()))
    B_C = np.nan_to_num(np.array(B.tolist()))
    return dot(A_C, B_C)/(norm(A_C)*norm(B_C))
    
def cosine_similarity_collaboration_filter(choose_movie_title, matrix, top_moive_title):
    result = []
    for mn in top_moive_title:
        # cosine similarity
        cor = cosine_similarity(matrix[choose_movie_title], matrix[mn])
        
        if np.isnan(cor):
            continue
        else:
            result.append((mn, '{:.2f}'.format(cor)))
            
    result.sort(key=lambda r: r[1], reverse=True)

    return result

choose_movie = [result[0] for result in pearson_collaboration_filter_result]
# choose_movie
### content base 를 통해 Pearson correlation 상위 10개의 가장 가까운 영화 찾기
cosine_similarity_collaboration_filter_result = cosine_similarity_collaboration_filter(choose_movie_title, matrix, choose_movie)
cosine_similarity_collaboration_filter_result

# for cm in choose_movie:
#     A = np.nan_to_num(np.array(matrix[choose_movie_title].tolist()))
#     B = np.nan_to_num(np.array(matrix[cm].tolist()))
#     print(dot(A, B), norm(A)*norm(B))

[('The Hours', '0.66'),
 ('Sissi', '0.66'),
 ('Rain Man', '0.64'),
 ('Monsoon Wedding', '0.62'),
 ('Back to the Future Part II', '0.59'),
 ('Trois couleurs : Rouge', '0.57'),
 ('Batman Returns', '0.56'),
 ('A Nightmare on Elm Street', '0.56'),
 ('Romeo + Juliet', '0.56'),
 ('Crustacés et coquillages', '0.55'),
 ('La science des rêves', '0.55'),
 ('Reservoir Dogs', '0.51'),
 ('Big Fish', '0.50'),
 ('Knallhart', '0.50'),
 ("Ocean's Eleven", '0.49'),
 ('Wag the Dog', '0.49'),
 ("La passion de Jeanne d'Arc", '0.49'),
 ('Titanic', '0.49'),
 ('Night on Earth', '0.48'),
 ('Aliens vs Predator: Requiem', '0.45'),
 ('Ariel', '0.43'),
 ('Monsieur Ibrahim et les fleurs du Coran', '0.42'),
 ('Star Wars', '0.41'),
 ('Say Anything...', '0.38'),
 ('Judgment Night', '0.38'),
 ('Armageddon', '0.37'),
 ('...Più forte ragazzi!', '0.37'),
 ('My Name Is Bruce', '0.37'),
 ('Die Höhle des gelben Hundes', '0.36'),
 ('Grbavica', '0.36'),
 ('When Saturday Comes', '0.35'),
 ('The Green Mile', '0.35'),
 ('Qui a tu